# 1. Imports and setup

In [1]:
#Import libraries

import matplotlib.pyplot as plt
import rasterio
from rasterio import plot
from rasterio.plot import show
from rasterio.windows import Window
from rasterio.transform import Affine
from PIL import Image
import numpy as np
import os
import torch
import torch.nn as nn

torch.random.manual_seed(0)

In [2]:
!nvidia-smi

NVIDIA-SMI has failed because you are not:
	a) running as an administrator or
	b) there is not at least one TCC device in the system



# 2. Preprocessing of images

In [3]:
input_dir = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\Imagenes\\Img\\input"
out_dir_allfiles = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\Imagenes\\Img\\output"
outNonDark = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\Imagenes\\Img\\NonDarkSplits"
outNonCloudy = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\Imagenes\\Img\\NonCloudySplits"

## 2.1. Cropping the splits

Geotiff images will be split in 512 by 512 tiles and PNGs

In [4]:
def pad_array(array, target_height, target_width):
    bands, height, width = array.shape
    padded = np.zeros((bands, target_height, target_width), dtype=array.dtype)
    padded[:, :height, :width] = array
    return padded

def normalize(rgb, clip_min=0, clip_max=3000):
    # Normalize each band individually
    rgb = np.clip(rgb, clip_min, clip_max)
    rgb = (rgb - clip_min) / (clip_max - clip_min)
    rgb = (rgb * 255).astype(np.uint8)
    return rgb

def split_geotiff_with_padding(input_dir, filename, output_dir, tile_size=512):
    os.makedirs(output_dir, exist_ok=True)
    png_dir = os.path.join(output_dir, "png")
    os.makedirs(png_dir, exist_ok=True)
    input_path = os.path.join(input_dir, filename)

    with rasterio.open(input_path) as src:
        width, height = src.width, src.height
        profile = src.profile

        for i in range(0, height, tile_size):
            for j in range(0, width, tile_size):
                window_height = min(tile_size, height - i)
                window_width = min(tile_size, width - j)
                window = Window(j, i, window_width, window_height)
                transform = src.window_transform(window)
                tile = src.read(window=window)

                if window_height < tile_size or window_width < tile_size:
                    tile = pad_array(tile, tile_size, tile_size)

                tile_profile = profile.copy()
                tile_profile.update({
                    'height': tile_size,
                    'width': tile_size,
                    'transform': transform
                })

                base_filename = os.path.splitext(filename)[0]
                tile_filename = f"{base_filename}_tile_{i // tile_size}_{j // tile_size}_p.tif"
                tile_path = os.path.join(output_dir, tile_filename)

                with rasterio.open(tile_path, 'w', **tile_profile) as dst:
                    dst.write(tile)

                # Generate PNG RGB
                if tile.shape[0] >= 3:
                    rgb = tile[:3]
                    rgb = normalize(rgb)
                    rgb = np.transpose(rgb, (1, 2, 0))  # Reorder to (H, W, Bands)
                    png_filename = f"{base_filename}_tile_{i//tile_size}_{j//tile_size}.png"
                    png_path = os.path.join(png_dir, png_filename)
                    Image.fromarray(rgb).save(png_path)


In [ ]:
# Loop through all .tif files in input_dir
for file in os.listdir(input_dir):
    if file.endswith('.TIF') or file.endswith('.tif'):
        # Create a directory for each tiff file in out_dir_allfiles
        base_filename = os.path.splitext(file)[0]
        output_dir = os.path.join(out_dir_allfiles, f"{base_filename}_tiles")
        os.makedirs(output_dir, exist_ok=True)
        
        # Call the custom function to generate tiles and PNGs
        split_geotiff_with_padding(input_dir, file, output_dir)

## 2.2. Filtering the tiles

As some tiles may include heavily clouded sectors or dark sectors, they must be excluded from the pool

In [5]:
#Function to filter the dark tiles
def is_nondark_tile(tile_path, dark_threshold=5, dark_ratio=0.3):
    """
    Determine if a tile is valid based on brightness and darkness thresholds.

    Parameters:
    - tile_path: Path to the GeoTIFF tile.
    - bright_threshold: Pixel value above which a pixel is considered bright.
    - dark_threshold: Pixel value below which a pixel is considered dark.
    - bright_ratio: Maximum allowed ratio of bright pixels.
    - dark_ratio: Maximum allowed ratio of dark pixels.

    Returns:
    - True if the tile is valid; False otherwise.
    """
    with rasterio.open(tile_path) as src:
        data = src.read()
        # Compute mean pixel value across all bands
        mean_pixel = np.mean(data, axis=0)
        total_pixels = mean_pixel.size
        # Calculate the fraction of dark pixels
        dark_pixels = np.sum(mean_pixel < dark_threshold)
        dark_fraction = dark_pixels / total_pixels
        # Determine validity based on threshold
        if dark_fraction > dark_ratio:
            return False
        return True

In [ ]:
for tiledir in os.listdir(out_dir_allfiles):
    for tile in os.listdir(os.path.join(out_dir_allfiles, tiledir)):
        if tile.endswith('.TIF') or tile.endswith('.tif'):
            tile_path = os.path.join(out_dir_allfiles, tiledir, tile)
            if is_nondark_tile(tile_path):
                if tiledir not in os.listdir(outNonDark):
                    os.makedirs(os.path.join(outNonDark, tiledir), exist_ok=True)
                # Copy the valid tile to the new directory just created (or existing)
                new_tile_path = os.path.join(outNonDark, tiledir)
                # copy "tile" file to "new_tile_path"
                os.system(f'copy "{tile_path}" "{new_tile_path}"')

In [17]:
upper_bound_ndvi = 1
lower_bound_ndvi = -0.3

def load_bands(filepath):
    with rasterio.open(filepath) as src:
        red = src.read(1).astype(np.float32)
        green = src.read(2).astype(np.float32)
        blue = src.read(3).astype(np.float32)
        nir = src.read(4).astype(np.float32)
    return red, green, blue, nir

def calculate_ndvi(nir, red):
    ndvi = (nir - red) / (nir + red + 1e-6)  # small epsilon to avoid division by zero
    ndvi = np.clip(ndvi, lower_bound_ndvi, upper_bound_ndvi)  #limit the values to the range [-1, 1]
    return ndvi

def detect_clouds(ndvi, threshold=0.1):
    cloud_mask = ndvi < threshold
    return cloud_mask

def cloud_coverage(cloud_mask):
    return np.mean(cloud_mask)

In [20]:
cloud_threshold = 0.07
coverage_threshold = 0.2
for tiledir in os.listdir(outNonDark):
    for tile in os.listdir(os.path.join(outNonDark, tiledir)):
        if tile.endswith('.TIF') or tile.endswith('.tif'):
            tile_path = os.path.join(outNonDark, tiledir, tile)
            red, green, blue, nir = load_bands(tile_path)
            ndvi = calculate_ndvi(nir, red)
            cloud_mask = detect_clouds(ndvi, cloud_threshold)
            coverage = cloud_coverage(cloud_mask)
            if coverage < coverage_threshold:
                if tiledir not in os.listdir(outNonCloudy):
                    os.makedirs(os.path.join(outNonCloudy, tiledir), exist_ok=True)
                # Copy the valid tile to the new directory just created (or existing)
                new_tile_path = os.path.join(outNonCloudy, tiledir)
                # copy "tile" file to "new_tile_path"
                os.system(f'copy "{tile_path}" "{new_tile_path}"')

## 2.3 Deleting the non RGB or NIR bands from Sentinel-2 tiles

In [6]:
inputTest = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\Imagenes\\Img\\ExampleTiles"
outputTest = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\Imagenes\\Img\\outputTest"

In [10]:
def preserve_first_four_bands_batch(input_dir, output_dir):
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Iterate through all .tif files in the input directory
    for filename in os.listdir(input_dir):
        if filename.endswith(".tif") and filename.startswith("S2"):
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_f.tif")

            with rasterio.open(input_path) as src:
                if src.count < 4:
                    continue

                # Update metadata
                meta = src.meta.copy()
                meta.update(count=4)

                with rasterio.open(output_path, 'w', **meta) as dst:
                    for i in range(1, 5):
                        dst.write(src.read(i), i)


In [8]:
preserve_first_four_bands_batch(inputTest, outputTest)

Processed: S2_Puno_20230302_tile_5_12_p.tif
Processed: S2_Puno_20230302_tile_5_13_p.tif
Processed: S2_Puno_20230302_tile_5_14_p.tif
Processed: S2_Puno_20230302_tile_5_15_p.tif
Processed: S2_Puno_20230302_tile_5_18_p.tif
Processed: S2_Puno_20230302_tile_5_19_p.tif
Processed: S2_Puno_20230302_tile_5_20_p.tif
Processed: S2_Puno_20230302_tile_6_6_p.tif
